# Общие замечания:

Во время проведения экспериментов по дообучению нейронных сетей, было выявлено, что связка <<обучение на metlin_8_25 - дообучение на hilic - тестирование на hilic_oxana>> показывает точность MAE на 2 меньше, чем  <<обучение на metlin_8_25 - дообучение на metlin_0_8 - тестирование на hilic_oxana>> (3 и 5 соответственно).

При сравнении наборов данных hilic и metlin были выявлены следующие различия:
- количество уникальных символов отличается
    - METLIN: 36
['#', '(', ')', '+', '-', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '@', 'B', 'C', 'F', 'H', 'I', 'N', 'O', 'P', 'S', '[', '\\', ']', 'c', 'i', 'l', 'n', 'o', 'r', 's']
    - HILIC: 29
['#', '(', ')', '+', '-', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'C', 'F', 'H', 'I', 'N', 'O', 'P', 'S', '[', ']', 'i', 'l', 'r']

- количество знаков после запятой в hilic до 6 знаков, в metlin - до 16 знаков

- элементы, состоящие из двух букв, которые встречаются в metlin {'Br', 'Cl', 'Si', 'No', 'Co', 'Cs'}, в hilic {'Br', 'Cl', 'Si'}

# Экперимент 1: замена строчных букв c и n на прописные C и N.

In [ ]:
metlin_0_8['smiles'] = metlin_0_8['smiles'].str.replace('c', 'C')
metlin_0_8['smiles'] = metlin_0_8['smiles'].str.replace('n', 'N')

In [ ]:
metlin_8_25['smiles'] = metlin_8_25['smiles'].str.replace('c', 'C')
metlin_8_25['smiles'] = metlin_8_25['smiles'].str.replace('n', 'N')

In [ ]:
metlin['smiles'] = metlin['smiles'].str.replace('c', 'C')
metlin['smiles'] = metlin['smiles'].str.replace('n', 'N')

# Экперимент 2:

сокращение количества знаков после запятой для значений времени удерживания в обоих наборах данных до 4.

In [ ]:
s = []
for i in metlin_8_25['rt']:
    s.append(len(str(i)))
print('Lengts of RT in metlin_8_25: ', set(s))

Lengts of RT in metlin_8_25:  {3, 4, 5, 6, 17, 18}


In [ ]:
s = []
for i in metlin_0_8['rt']:
    s.append(len(str(i)))
print('Lengts of RT in metlin_0_8: ', set(s))

Lengts of RT in metlin_0_8:  {3, 4, 5, 17, 18}


In [ ]:
s = []
for i in hilic['rt']:
    s.append(len(str(i)))
print('Lengts of RT in hilic: ', set(s))

Lengts of RT in hilic:  {4, 5, 6, 7, 8, 9}


In [ ]:
metlin_0_8.loc[:, 'rt'] = metlin_0_8['rt'].apply(lambda x: float('{:.4f}'.format(x)))
metlin_8_25.loc[:, 'rt'] = metlin_8_25['rt'].apply(lambda x: float('{:.4f}'.format(x)))
hilic.loc[:, 'rt'] = hilic['rt'].apply(lambda x: float('{:.4f}'.format(x)))

# Экперимент 3:

замена названий элементов, состоящих из 2 букв на символы {'Br': '*', 'Cl': '&', 'Si': '%', 'Sc': '!', 'Cn': '?'}

In [ ]:
elements_symbols = [
    "H", "He", "Li", "Be", "B", "C",
    "N", "O", "F", "Ne", "Na", "Mg",
    "Al", "Si", "P", "S", "Cl", "Ar",
    "K", "Ca", "Sc", "Ti", "V", "Cr",
    "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
    "Ga", "Ge", "As", "Se", "Br", "Kr",
    "Rb", "Sr", "Y", "Zr", "Nb", "Mo",
    "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",
    "In", "Sn", "Sb", "Te", "I", "Xe",
    "Cs", "Ba", "La", "Ce", "Pr", "Nd",
    "Pm", "Sm", "Eu", "Gd", "Tb", "Dy",
    "Ho", "Er", "Tm", "Yb", "Lu", "Hf",
    "Ta", "W", "Re", "Os", "Ir", "Pt",
    "Au", "Hg", "Tl", "Pb", "Bi", "Po",
    "At", "Rn", "Fr", "Ra", "Ac", "Th",
    "Pa", "U", "Np", "Pu", "Am", "Cm",
    "Bk", "Cf", "Es", "Fm", "Md", "No",
    "Lr", "Rf", "Db", "Sg", "Bh", "Hs",
    "Mt", "Ds", "Rg", "Cn", "Nh", "Fl",
    "Mc", "Lv", "Ts", "Og"
]

In [ ]:
res = []
for i in metlin['smiles']:
    for j in elements_symbols:
      if j in str(i) and len(j) == 2:
          res.append(j)
print(set(res))

{'Br', 'Cl', 'Si', 'No', 'Co', 'Cs'}


In [ ]:
res = []
for i in hilic['smiles']:
    for j in elements_symbols:
      if j in str(i) and len(j) == 2:
          res.append(j)
print(set(res))

{'Br', 'Cl', 'Si'}


In [ ]:
replacements = {'Br': '*', 'Cl': '&', 'Si': '%', 'Sc': '!', 'Cn': '?'}

def replace_symbols(smiles):
    for old, new in replacements.items():
        smiles = smiles.replace(old, new)
    return smiles

metlin_0_8['smiles'] = metlin_0_8['smiles'].apply(replace_symbols)
metlin_8_25['smiles'] = metlin_8_25['smiles'].apply(replace_symbols)
hilic['smiles'] = hilic['smiles'].apply(replace_symbols)
metlin['smiles'] = metlin['smiles'].apply(replace_symbols)

Из-за сокращения длины всех строк smiles на 1 символ, необходимо снова выравнивать длину каждой строки до 185 символов.

In [ ]:
maxlen = 185

for i in range(79957):
    if len(metlin['smiles'].values[i]) < maxlen:
        metlin['smiles'].values[i] = metlin['smiles'].values[i].ljust(maxlen)

for i in range(2068):
    if len(metlin_0_8['smiles'].values[i]) < maxlen:
        metlin_0_8['smiles'].values[i] = metlin_0_8['smiles'].values[i].ljust(maxlen)

for i in range(77889):
    if len(metlin_8_25['smiles'].values[i]) < maxlen:
        metlin_8_25['smiles'].values[i] = metlin_8_25['smiles'].values[i].ljust(maxlen)

for i in range(880):
    if len(hilic['smiles'].values[i]) < maxlen:
        hilic['smiles'].values[i] = hilic['smiles'].values[i].ljust(maxlen)